In [34]:
import numpy as np
from Utils.Toolkit import get_data, generate_SM_user_feature_matrix, get_features_ratings

In [35]:
data = get_data()
URM_train = data['train'].tocsr()
ICM = data['ICM_subclass'].tocsr()
UFM = generate_SM_user_feature_matrix(URM_train, ICM).tocsr()

Evaluating SM_user_feature_matrix: 100%|██████████| 30911/30911 [00:10<00:00, 3069.21it/s]


Generated UFM with shape (30911, 2011)


In [36]:
URM_train

<30911x18495 sparse matrix of type '<class 'numpy.float64'>'
	with 371381 stored elements in Compressed Sparse Row format>

In [37]:
ICM

<18495x2011 sparse matrix of type '<class 'numpy.float64'>'
	with 18495 stored elements in Compressed Sparse Row format>

In [38]:
most_interactions = 0
user = 0

for i in range(URM_train.shape[0]):
    start_pos = URM_train.indptr[i]
    end_pos = URM_train.indptr[i+1]
    
    if len(URM_train.data[start_pos:end_pos]) > most_interactions:
        most_interactions = len(URM_train.data[start_pos:end_pos])
        user = i

print(f'User = {user} -> interacted = {most_interactions}')

User = 63 -> interacted = 676


In [39]:
start_pos = URM_train.indptr[user]
end_pos = URM_train.indptr[user+1]
mask = URM_train.indices[start_pos:end_pos]
user_profile = np.squeeze(np.asarray(ICM[mask,:].sum(axis=0)))
user_profile

array([0., 2., 0., ..., 0., 0., 0.])

In [40]:
values = user_profile > 0
user_profile

array([0., 2., 0., ..., 0., 0., 0.])

In [41]:
SM_user_feature_matrix = np.zeros((5, 5), dtype=int)
SM_user_feature_matrix[0] = user_profile[:5]
SM_user_feature_matrix

array([[0, 2, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [42]:
rows = []
rows.append(np.arange(10))
rows.append(np.arange(10))
np.asarray(rows).ravel()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [43]:
start_pos = UFM.indptr[63]
end_pos = UFM.indptr[63+1]
non_zero = UFM.data[start_pos:end_pos] > 0
UFM[63].toarray().ravel()[non_zero]

array([ 2,  1,  2,  2,  2,  1,  1,  1,  2,  1,  1,  3,  1,  1,  2,  1,  1,
        2,  2,  1,  1,  5, 10,  1,  1,  5,  1,  2,  1,  1,  1,  2,  6,  1,
        1,  1,  1,  3,  1,  2,  2,  1,  1,  2,  1,  1,  1,  1,  1,  3,  1,
        7,  8,  2,  2,  9,  1,  2,  3,  1,  1,  2,  2,  2,  2,  7,  2,  2,
        1,  3,  1,  1,  3,  1,  1,  1,  1,  2,  3,  3,  2,  2,  8,  3,  1,
        1,  3,  1,  4,  1,  5,  1,  1,  1,  1,  1,  1,  5,  1,  1,  2,  1,
        3,  2,  1,  2,  1,  1,  2,  1,  1,  2,  1,  1,  7,  7,  1,  1,  1,
        3,  1,  4,  4,  3,  1,  1,  2,  1,  1,  1,  1,  1,  1,  3,  2,  1,
        1,  1,  1,  3,  2,  1,  1,  1,  1,  2,  1,  1,  1,  1,  1, 12,  1,
        1,  2,  1,  1,  1,  2,  2,  1,  2,  2,  2,  1,  4,  5,  1,  3,  1,
        6,  1,  1,  1,  1,  3,  2,  4,  3,  1,  1,  2,  3,  2,  1,  1,  1,
        1,  1,  1,  3, 11,  2,  1,  2,  1,  1,  5,  1,  3,  1,  1,  2,  1,
        2,  1,  2,  1,  2,  1,  3,  1,  1,  1,  1,  1,  4,  1,  1,  1,  1,
        1,  1,  2,  1,  1

In [44]:
recommended_features_indexes = np.flip(np.argsort(UFM[63].toarray().ravel()), 0)
ordered_features = UFM[63].toarray().ravel()[recommended_features_indexes]
ordered_features

array([17, 14, 12, ...,  0,  0,  0])

In [45]:
ordered_features_mul = np.where(ordered_features > 0, 1, ordered_features)
ordered_features_mul

array([1, 1, 1, ..., 0, 0, 0])

In [46]:
recommended_features = np.zeros(UFM.shape[1])
recommended_features = recommended_features + recommended_features_indexes
recommended_features *= ordered_features_mul
recommended_features

array([1718., 1581.,  985., ...,    0.,    0.,    0.])

In [47]:
max(UFM[63].data)

17

In [49]:
RM_features, RM_ratings = get_features_ratings(URM_train, ICM, at=10)
RM_features

Evaluating SM_user_feature_matrix: 100%|██████████| 30911/30911 [00:09<00:00, 3256.05it/s]


Generated UFM with shape (30911, 2011)


<30911x10 sparse matrix of type '<class 'numpy.int64'>'
	with 168721 stored elements in Compressed Sparse Row format>

In [57]:
RM_features[4].data

array([1581,  815,  766,  721,  839, 1645])

In [58]:
RM_ratings[4].data

array([2, 1, 1, 1, 1, 1])

In [59]:
np.ediff1d(RM_ratings[4].data)

array([-1,  0,  0,  0,  0])

In [60]:
rank = [1]
for i in range(len(RM_features[4].data) - 1):
    if np.ediff1d(RM_ratings[4].data)[i] == 0:
        rank.append(rank[i])
    else:
        rank.append(rank[i] + 1)

rank = np.array(rank)
rank

array([1, 2, 2, 2, 2, 2])

In [61]:
URM_train[4].indices

array([ 543,  663,  666, 1172, 4657, 4658, 9695], dtype=int32)

In [62]:
user = 4
s_p = URM_train.indptr[user]
e_p = URM_train.indptr[user+1]
print(URM_train.data[s_p:e_p])
URM_train.data[s_p:e_p] += 3
print(URM_train.data[s_p:e_p])

[1. 1. 1. 1. 1. 1. 1.]
[4. 4. 4. 4. 4. 4. 4.]
